# Creating and modelling metallic supercells

In this section we will be concerned with modelling supercells of aluminium.
When dealing with periodic problems there is no unique definition of the
lattice: Clearly any duplication of the lattice along an axis is also a valid
repetitive unit to describe exactly the same system.
This is exactly what a **supercell** is: An $n$-fold repetition along one of the
axes of the original lattice.

The following code achieves this for aluminium:

In [1]:
using DFTK
using LinearAlgebra
using ASEconvert

function aluminium_setup(repeat=1; Ecut=7.0, kgrid=[2, 2, 2])
    a = 7.65339
    lattice = a * Matrix(I, 3, 3)
    Al = ElementPsp(:Al; psp=load_psp("hgh/lda/al-q3"))
    atoms     = [Al, Al, Al, Al]
    positions = [[0.0, 0.0, 0.0], [0.0, 0.5, 0.5], [0.5, 0.0, 0.5], [0.5, 0.5, 0.0]]
    unit_cell = periodic_system(lattice, atoms, positions)

    # Make supercell in ASE:
    # We convert our lattice to the conventions used in ASE, make the supercell
    # and then convert back ...
    supercell_ase = convert_ase(unit_cell) * pytuple((repeat, 1, 1))
    supercell     = pyconvert(AbstractSystem, supercell_ase)

    # Unfortunately right now the conversion to ASE drops the pseudopotential information,
    # so we need to reattach it:
    supercell = attach_psp(supercell; Al="hgh/lda/al-q3")

    # Construct an LDA model and discretise
    # Note: We disable symmetries explicitly here. Otherwise the problem sizes
    #       we are able to run on the CI are too simple to observe the numerical
    #       instabilities we want to trigger here.
    model = model_LDA(supercell; temperature=1e-3, symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid)
end;

    CondaPkg Found dependencies: /home/runner/.julia/packages/ASEconvert/CNQ1A/CondaPkg.toml
    CondaPkg Found dependencies: /home/runner/.julia/packages/PythonCall/wXfah/CondaPkg.toml
    CondaPkg Dependencies already up to date


As part of the code we are using a routine inside the ASE,
the [atomistic simulation environment](https://wiki.fysik.dtu.dk/ase/index.html)
for creating the supercell and make use of the two-way interoperability of
DFTK and ASE. For more details on this aspect see the documentation
on Input and output formats.

Write an example supercell structure to a file to plot it:

In [2]:
setup = aluminium_setup(5)
convert_ase(periodic_system(setup.model)).write("al_supercell.png")

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123


Python: None

<img src="https://docs.dftk.org/stable/examples/al_supercell.png" width=500 height=500 />

As we will see in this notebook the modelling of a system generally becomes
harder if the system becomes larger.

- This sounds like a trivial statement as *per se* the cost per SCF step increases
  as the system (and thus $N$) gets larger.
- But there is more to it:
  If one is not careful also the *number of SCF iterations* increases
  as the system gets larger.
- The aim of a proper computational treatment of such supercells is therefore
  to ensure that the **number of SCF iterations remains constant** when the
  system size increases.

For achieving the latter DFTK by default employs the `LdosMixing`
preconditioner [^HL2021] during the SCF iterations. This mixing approach is
completely parameter free, but still automatically adapts to the treated
system in order to efficiently prevent charge sloshing. As a result,
modelling aluminium slabs indeed takes roughly the same number of SCF iterations
irrespective of the supercell size:

[^HL2021]:
   M. F. Herbst and A. Levitt.
   *Black-box inhomogeneous preconditioning for self-consistent field iterations in density functional theory.*
   J. Phys. Cond. Matt *33* 085503 (2021). [ArXiv:2009.01665](https://arxiv.org/abs/2009.01665)

In [3]:
self_consistent_field(aluminium_setup(1); tol=1e-4);

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -8.298561930768                   -0.85    5.2         
  2   -8.300220072859       -2.78       -1.25    1.0    112ms
  3   -8.300438761612       -3.66       -1.89    3.1    147ms
  4   -8.300461892560       -4.64       -2.79    1.8    122ms
  5   -8.300464536514       -5.58       -3.16    3.0    246ms
  6   -8.300464607134       -7.15       -3.34    4.4    146ms
  7   -8.300464626074       -7.72       -3.49    2.4    128ms
  8   -8.300464635888       -8.01       -3.65    1.2    103ms
  9   -8.300464641052       -8.29       -3.81    4.0    169ms
 10   -8.300464642660       -8.79       -3.93    2.1    162ms
 11   -8.300464643777       -8.95       -4.22    1.5    117ms


In [4]:
self_consistent_field(aluminium_setup(2); tol=1e-4);

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -16.63742580205                   -0.71    6.2         
  2   -16.67682217374       -1.40       -1.14    1.4    274ms
  3   -16.67915600275       -2.63       -1.87    3.8    337ms
  4   -16.67926678130       -3.96       -2.57    3.6    348ms
  5   -16.67928547672       -4.73       -3.13    3.0    359ms
  6   -16.67928619003       -6.15       -3.48    4.4    353ms
  7   -16.67928620778       -7.75       -4.00    2.2    285ms


In [5]:
self_consistent_field(aluminium_setup(4); tol=1e-4);

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -33.32557637554                   -0.56    6.9         
  2   -33.33251396177       -2.16       -1.00    1.0    953ms
  3   -33.33408966324       -2.80       -1.74    4.4    1.25s
  4   -33.33567869872       -2.80       -2.24    3.1    1.12s
  5   -33.33690239531       -2.91       -2.47   12.2    1.82s
  6   -33.33693069752       -4.55       -2.75    4.2    1.08s
  7   -33.33694354811       -4.89       -3.59    3.1    1.19s
  8   -33.33694362180       -7.13       -3.70    4.5    1.37s
  9   -33.33694375878       -6.86       -4.17    1.4    909ms


When switching off explicitly the `LdosMixing`, by selecting `mixing=SimpleMixing()`,
the performance of number of required SCF steps starts to increase as we increase
the size of the modelled problem:

In [6]:
self_consistent_field(aluminium_setup(1); tol=1e-4, mixing=SimpleMixing());

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -8.297948166808                   -0.85    5.2         
  2   -8.300240735629       -2.64       -1.58    1.0   82.2ms
  3   -8.300333538160       -4.03       -2.22    3.1    118ms
  4   -8.300309951757   +   -4.63       -2.16    2.2    113ms
  5   -8.300463618910       -3.81       -3.44    1.0   80.1ms
  6   -8.300464167966       -6.26       -3.46    6.8    193ms
  7   -8.300464592357       -6.37       -3.87    1.0   85.0ms
  8   -8.300464636279       -7.36       -4.16    1.9    122ms


In [7]:
self_consistent_field(aluminium_setup(4); tol=1e-4, mixing=SimpleMixing());

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -33.32396412968                   -0.56    7.8         
  2   -33.29113566406   +   -1.48       -1.25    1.9    857ms
  3   +6.031103801273   +    1.59       -0.27    6.6    2.01s
  4   -33.30945504430        1.59       -1.57    5.6    1.72s
  5   -33.12976735985   +   -0.75       -1.28    4.8    1.37s
  6   -28.82950944590   +    0.63       -0.74    4.5    1.41s
  7   -33.32028183455        0.65       -1.87    4.9    1.35s
  8   -33.33464671140       -1.84       -2.18    2.6    1.06s
  9   -33.33510636581       -3.34       -2.15    2.8    1.20s
 10   -33.33533850237       -3.63       -2.45    1.8    967ms
 11   -33.33674353031       -2.85       -2.82    2.0    939ms
 12   -33.33679303442       -4.31      

For completion let us note that the more traditional `mixing=KerkerMixing()`
approach would also help in this particular setting to obtain a constant
number of SCF iterations for an increasing system size (try it!). In contrast
to `LdosMixing`, however, `KerkerMixing` is only suitable to model bulk metallic
system (like the case we are considering here). When modelling metallic surfaces
or mixtures of metals and insulators, `KerkerMixing` fails, while `LdosMixing`
still works well. See the Modelling a gallium arsenide surface example
or [^HL2021] for details. Due to the general applicability of `LdosMixing` this
method is the default mixing approach in DFTK.